# Deep Image Stack

![](./assets/2024-06-24_DeepImageStack.png)

## Overview (WIP)
This workflow covers navigating a deep image stack. Often each subsequent frame corresponds to a concurrent time sample, such as the case in neuroscience when conducting calcium imaging with a microscope.



## Prerequisites and Resources (WIP)

| Topic | Type | Notes |
| --- | --- | --- |

## Imports and Configuration

In [ ]:
from pathlib import Path
import numpy as np
import xarray as xr
import holoviews as hv
from holoviews.streams import Stream
from holoviews.operation.datashader import rasterize
from hvplot import xarray
import panel as pn

pn.extension(throttled=True)
hv.extension('bokeh')

## Loading and Inspecting the Data

Let's read the data in chunks, emulating a situation where a dataset is too large to fit into memory. Utilizing the `chunks` parameter in `xr.open_dataset` is crucial for efficient data handling with large datasets, as it enables Dask to process the data in manageable portions.

In [ ]:
DATA_PATH = "data/real_miniscope_uint8.zarr"
DATA_PATH = "data/sim_miniscope.zarr"

# Open the dataset from the Zarr storage
ds = xr.open_dataset(
    DATA_PATH,
    engine = 'zarr',
    chunks = {'frame': 'auto', 'height':-1, 'width':-1},  # chunk by sets of complete frames
)
ds

From the output above, we can see that the actual DataArray that we are looking for is called '`varr_ref` - let's go ahead and get a handle on that.

In [ ]:
da = ds['sim_miniscope']
da

## Data Visualization

Visualizing calcium imaging data effectively is key to extracting meaningful insights. We introduce various visualization approaches to cater to different analysis needs.

We will start with one-liner viewer and then proceed to a more advanced application with enhanced interactive features and exposed controls, at the expense of code complexity.

### Basic: Quick App with Time-Projected Image

We can use one line with hvPlot for a quick inspection of the deep image stack. To the left of this, we'll also put a max-projection of the image stack frame over time as a reference of all potential neurons that fluorescence at some point in the movie. Max projecting over time just means keeping the maximum value from the time-stack in each position of the width/height coordinates.

In [ ]:
# Create the max time-projected image
max_proj_time = da.max('frame').compute().astype(np.float32)
img_max_proj_time = max_proj_time.hvplot.image(x='width', y='height', title='Max Over Time', cmap = "magma", aspect = da.sizes['width'] / da.sizes['height'],)

# Create the Deep Image Stack App (this line is sufficient to create a basic app)
img_stack_viewer = da.hvplot.image(groupby="frame", title='Deep Image Stack', cmap = "viridis", aspect = da.sizes['width'] / da.sizes['height'])

basic_img_stack_app = img_max_proj_time + img_stack_viewer
basic_img_stack_app

This was a quick way to see one frame at a time! But it looks like there are a lot of fluorescing neurons in the left `'Max Over Time'` image that came from somewhere in the data. How do we visually locate which frames in the `'Deep Image Stack'` (right) a neuron is flourescing in?

Well, our data array is a three-dimensional volume, so if we also had **side-view** of the volume, we might be able to locate the frames where a particular region is fluorescing.

### Advanced App with Side-Views, Swipe Overlay, and Annotations Linked to Timeseries View

The Side-View App builds on the Basic App with added functionality:

1. **Continuous Playback:** Player widget for continuous playback, along with controls for step-by-step examination of the image stack.
2. **Side Views** Aggregated side views for display over 'deep' dimension.
3. **Synchronized Frame Indicators** Frame markers synchronized with the playback and x,y range of the main image stack view.
4. **Swipe Overlay** Draggable swipe interaction on the main plot to display additional overlay view and direct comparison.
5. TODO: **Interactive Annotations** Enables direct annotation of regions of interest within the visual interface utilizing the HoloNote package.
6. TODO: **Timeseries of Annotations** Annotations are synchronized to an adjacent stacked timeseries plot.

In [ ]:
hv.Curve([]).opts(backend_opts={'plot.toolbar_sticky':False})

In [ ]:
def plot_image(value):
    return hv.Image(da.sel(frame=value), kdims=["width", "height"]).opts(
        frame_height=da.sizes['height'],
        frame_width=da.sizes['width'],
        cmap = "Viridis",
        title = "⬅️ Deep Image Stack || Max Over Time ➡️",
        tools=['hover', 'crosshair'],
        toolbar='below',
        backend_opts={'plot.toolbar_sticky':False}
    )

# Create a player widget
video_player = pn.widgets.Player(
    length =len(da.coords["frame"]),
    interval = 250,  # ms
    value = 950, # start frame
    width=da.sizes['width'],
    height=90,
    loop_policy="loop",
)

# Create the main frame-wise view (height by width)
main_view = hv.DynamicMap(pn.bind(plot_image, video_player))

# right-side view: frame by height
right_data = da.mean(["width"]).persist()
right_view = rasterize(
    hv.Image(right_data, kdims=["frame", "height"]).opts(
        frame_height=da.sizes['height'],
        frame_width=175,
        colorbar=False,
        title="Side View",
        toolbar=None,
        cmap = "Viridis",
        tools=['hover', 'crosshair'],
        yaxis='right',
        # xlabel='frame',
        # ylabel='height',
    )#.redim(frame='_frame', height = '_height') # redim to unlink with main
)


# top-side view: width by frame 
top_data = da.mean(["height"]).persist()
top_view = rasterize(
    hv.Image(top_data, kdims=["width", "frame"]).opts(
        frame_height=175,
        frame_width=da.sizes['width'],
        colorbar=False,
        toolbar=None,
        cmap = "Viridis",
        title= "Top View",
        tools=['hover', 'crosshair'],
        xaxis='top',
        # xlabel='width',
        # ylabel='frame',
    )#.redim(frame='_frame', width = '_width') # redim to unlink with main
)

# frame-indicator lines on side view plots
line_opts = dict(color="red", line_width=7, line_alpha=.4)

xyrange_stream = hv.streams.RangeXY(source=main_view)

def plot_hline(value, x_range, y_range):
    if x_range == None:
        x_range = [int(da.width[0].values), int(da.width[-1].values)]
    return hv.Segments((x_range[0], value, x_range[1], value))

def plot_vline(value, x_range, y_range):
    if y_range == None:
        y_range = [int(da.height[0].values), int(da.height[-1].values)]
    return hv.Segments((value, y_range[0], value, y_range[1]))

dmap_hline = hv.DynamicMap(pn.bind(plot_hline, video_player), streams=[xyrange_stream]).opts(
    **line_opts
)

dmap_vline = hv.DynamicMap(pn.bind(plot_vline, video_player), streams=[xyrange_stream]).opts(
    **line_opts
)

# Create the max time-projected image
max_proj_time = da.max('frame').compute().astype(np.float32)
img_max_proj_time = hv.Image(
    max_proj_time, ['width', 'height'], label='Max Over Time').opts(
    frame_height=da.sizes['height'],
    frame_width=da.sizes['width'],
    cmap='magma',
    tools=['hover', 'crosshair'],
    toolbar='right',
    title = "⬅️ Deep Image Stack || Max Over Time ➡️",
)

# Bind the player widget's value directly to the markdown pane's object parameter
frame_markdown = pn.pane.Markdown()
frame_markdown.object = pn.bind(lambda value: f'# frame: {value}', video_player.param.value)

# Lay out the app views
video_player.margin = (20, 20, 20, 70)  # center widget over main
img_stack_app = pn.Column(
    top_view * dmap_hline,
    pn.Row(pn.Swipe(main_view, img_max_proj_time, value=55, max_height=100, slider_color='grey'),
           right_view * dmap_vline), 
    pn.Row(video_player, frame_markdown)
)

img_stack_app.servable()


In [ ]:
def plot_image(value):
    return hv.Image(da.sel(frame=value), kdims=["width", "height"]).opts(
        frame_height=da.sizes['height'],
        frame_width=da.sizes['width'],
        cmap = "Viridis",
        # title = "Deep Image Stack [Max Over Time]",
        tools=['hover', 'crosshair'],
        toolbar='right',
        backend_opts={'plot.toolbar_sticky':False}
    )

# Create a player widget
video_player = pn.widgets.Player(
    length =len(da.coords["frame"]),
    interval = 250,  # ms
    value = 950, # start frame
    width=da.sizes['width'],
    height=90,
    loop_policy="loop",
)

# Create the main frame-wise view (height by width)
main_view = hv.DynamicMap(pn.bind(plot_image, video_player))

# right-side view: frame by height
right_data = da.mean(["width"]).persist()
right_view = rasterize(
    hv.Image(right_data, kdims=["frame", "height"]).opts(
        frame_height=da.sizes['height'],
        frame_width=175,
        colorbar=False,
        title="Side View",
        toolbar=None,
        cmap = "Viridis",
        tools=['hover', 'crosshair'],
        yaxis='right',
        # xlabel='frame',
        # ylabel='height',
    )#.redim(frame='_frame', height = '_height') # redim to unlink with main
)


# top-side view: width by frame 
top_data = da.mean(["height"]).persist()
top_view = rasterize(
    hv.Image(top_data, kdims=["width", "frame"]).opts(
        frame_height=175,
        frame_width=da.sizes['width'],
        colorbar=False,
        toolbar=None,
        cmap = "Viridis",
        title= "Top View",
        tools=['hover', 'crosshair'],
        xaxis='top',
        # xlabel='width',
        # ylabel='frame',
    )#.redim(frame='_frame', width = '_width') # redim to unlink with main
)

# frame-indicator lines on side view plots
line_opts = dict(color="red", line_width=7, line_alpha=.4)

xyrange_stream = hv.streams.RangeXY(source=main_view)

def plot_hline(value, x_range, y_range):
    if x_range == None:
        x_range = [int(da.width[0].values), int(da.width[-1].values)]
    return hv.Segments((x_range[0], value, x_range[1], value))

def plot_vline(value, x_range, y_range):
    if y_range == None:
        y_range = [int(da.height[0].values), int(da.height[-1].values)]
    return hv.Segments((value, y_range[0], value, y_range[1]))

dmap_hline = hv.DynamicMap(pn.bind(plot_hline, video_player), streams=[xyrange_stream]).opts(
    **line_opts
)

dmap_vline = hv.DynamicMap(pn.bind(plot_vline, video_player), streams=[xyrange_stream]).opts(
    **line_opts
)

# Create the max time-projected image
max_proj_time = da.max('frame').compute().astype(np.float32)
img_max_proj_time = hv.Image(
    max_proj_time, ['width', 'height'], label='Max Over Time').opts(
    frame_height=da.sizes['height'],
    frame_width=da.sizes['width'],
    cmap='magma',
    # tools=['hover', 'crosshair'],
    # toolbar='right',
)

# Bind the player widget's value directly to the markdown pane's object parameter
frame_markdown = pn.pane.Markdown()
frame_markdown.object = pn.bind(lambda value: f'# frame: {value}', video_player.param.value)

alpha_slider = pn.widgets.FloatSlider(start=0, end=1, step=.001, value=0.5, name='Alpha of Max Over Time', align='center')

# Lay out the app views
video_player.margin = (0, 0, 0, -70)  # center widget with main
alpha_slider.margin = (0, 0, 20, -170)  # center widget with main
img_stack_app = pn.Column(
    top_view * dmap_hline,
    pn.Row(
        main_view * img_max_proj_time.apply.opts(alpha=alpha_slider),
           right_view * dmap_vline, margin=0), 
    pn.Column(
    alpha_slider,
    pn.Row(video_player, frame_markdown), align='center')
)

img_stack_app.servable()


# With Annotations (WIP)

In [ ]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
# from holonote.app.tabulator import AnnotatorTabulator


# TODO: Create a secondary subcoordinate_y plot that is synced with the holonote table of annotations


# # Update plot based on annotation selection
# def plot_timeseries_by_select(indices):
#     if indices:
#         h1, h2, w1, w2 = indices[0]["start[height]"], indices[0]["end[height]"], indices[0]["start[width]"], indices[0]["end[width]"]
#         ds_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2)).mean(
#             ["height", "width"]
#         )
#         time_series.object = hv.Curve(ds_sel)
#         )

# Update plot based on annotation creation
def plot_timeseries_by_stream(bounds):
    if bounds:
        h1, h2, w1, w2 = bounds
        ds_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
        time_series.object = hv.Curve(ds_sel.mean(["height", "width"])).opts(
        )

# Annotation setup
annotator = Annotator({"height": float, "width": float}, fields=["type"])
annotator.groupby = "type"
annotator_widgets = PanelWidgets(annotator) #, AnnotatorTabulator(annotator))

time_series = pn.pane.HoloViews()

# # Update timeseries plot on annotation selection
# pn.bind(plot_timeseries_by_select, annotator.param.selected_indices, watch=True)

# Update timeseries plot on annotation creation
display = annotator.get_display("height", "width")
box_stream = display._edit_streams[0]
box_stream.source = main_view
pn.bind(plot_timeseries_by_stream, box_stream.param.bounds, watch=True)

img_stack_app_annotator = pn.Column(annotator_widgets,
    annotator * top_view * dmap_hline,
    pn.Row(img_max_proj_time * annotator, # TODO: file bug about Swipe causing frame_width issue
           annotator * right_view * dmap_vline, 
          ), 
    pn.Row(video_player, frame_markdown),
    # time_series
)

img_stack_app_annotator.servable()

Issues:
- Creating an annotation: AssertionError: DynamicMap must only contain one type of object, not both Overlay and NdOverlay.
- Visibility GUI not updating
- Side View not showing annotation on correct axis
- multiple toolbars with swipe (causing frame misalignment)
- how to show multiple colorbars with swipe?
- add linked timeseries plot
- freehanddraw polygon to box annotation, with polygon saved as a displayable field?